In [ ]:
from function import *

In [ ]:
print(os.getcwd())
os.chdir("/home/wmbio/WORK/gitworking/Dependency_prediction/")
print(os.getcwd())

## **Data Load & Path**

In [ ]:
TRAIN_PATH = "/home/wmbio/WORK/gitworking/Dependency_prediction/preprocessing/DATA/2022-07-28/"
TEMP_PATH = "prediction/train_preprocessing/"
SAVE_PATH = "prediction/custom_model/"
CHECK_PATH = SAVE_PATH + "checkpoint/"
PREMODEL_PATH = "prediction/data/premodel_tcga/"

* **All Omics**

In [ ]:
data_exp, data_mut, data_cna, data_meth, data_fprint, data_dep = trainset_load()
gc.collect()

# paper
# with open('prediction/data/paper/ccl_complete_data_278CCL_1298DepOI_360844samples.pickle', 'rb') as f:
#     data_mut, data_exp, data_cna, data_meth, data_dep, data_fprint = pickle.load(f)

## **Build Model**

* **Pretrained Model load - TCGA**

In [ ]:
premodel_mut = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_mut_1000_100_50.pickle', 'rb'))
premodel_exp = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_exp_500_200_50.pickle', 'rb'))
premodel_cna = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_cna_500_200_50.pickle', 'rb'))
premodel_meth = pickle.load(open(PREMODEL_PATH + 'premodel_tcga_meth_500_200_50.pickle', 'rb'))
print("\n\nDatasets successfully loaded.")

* **Golbal variable**

In [ ]:
batch_size = 500
num_epoch = 100
num_DepOI = 1298 # 1298 DepOIs as defined in our paper, custom 1227
num_ccl = int(data_exp.shape[0]/num_DepOI)
split_ratio = 0.7

* **Train / Test split**

In [ ]:
# 80% CCLs for training/validation, and 20% for testing
id_rand = np.random.permutation(num_ccl)
id_cell_train = id_rand[np.arange(0, round(num_ccl * split_ratio))]
id_cell_test = id_rand[np.arange(round(num_ccl * split_ratio), num_ccl)]
id_train = np.arange(0, num_DepOI) + id_cell_train[0]*num_DepOI
for y in id_cell_train:
    id_train = np.union1d(id_train, np.arange(0, num_DepOI) + y*num_DepOI)
id_test = np.arange(0, num_DepOI) + id_cell_test[0] * num_DepOI
for y in id_cell_test:
    id_test = np.union1d(id_test, np.arange(0, num_DepOI) + y*num_DepOI)
print("\n\nTraining/validation on %d samples (%d CCLs x %d DepOIs) and testing on %d samples (%d CCLs x %d DepOIs).\n\n" % (
    len(id_train), len(id_cell_train), num_DepOI, len(id_test), len(id_cell_test), num_DepOI))

## **Fit & Evaluation Model**

### **(1) Full Model 4-Omics**

In [ ]:
with tf.device('/cpu:0'):
    K.clear_session()
    model = full_model(data_mut=data_mut, data_exp=data_exp, data_cna=data_cna,data_meth=data_meth,
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp, premodel_cna=premodel_cna,
                      premodel_meth=premodel_meth, save_path = CHECK_PATH)

In [ ]:
SAVE_NAME = "model_full_0801"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(2) Expression-Mutation-CNA Model**

In [ ]:
with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_exp_cna_model(data_mut=data_mut, data_exp=data_exp, data_cna=data_cna,
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp, premodel_cna=premodel_cna,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1])

In [ ]:
SAVE_NAME = "model_mut_exp_cna_0801"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMut_Exp_CNA-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(3) Expression-Mutation Model**

In [ ]:
with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_exp_model(data_mut=data_mut, data_exp=data_exp, 
                      data_fprint=data_fprint, data_dep=data_dep, id_train=id_train, id_test=id_test,
                      premodel_mut=premodel_mut, premodel_exp=premodel_exp,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1])

In [ ]:
SAVE_NAME = "model_mut_exp_0801"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nMut_CNA-DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(4) Expression Model**

In [ ]:
with tf.device('/cpu:0'):
    K.clear_session()
    model = exp_model(data_exp=data_exp, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_exp=premodel_exp,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1])

In [ ]:
SAVE_NAME = "model_exp_0801"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(5) Mutation Model**

In [ ]:
with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_model(data_mut=data_mut, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_exp=premodel_exp,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1])

In [ ]:
SAVE_NAME = "model_mut_0801"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))

### **(6) CNA Model**

In [ ]:
with tf.device('/cpu:0'):
    K.clear_session()
    model = mut_model(data_mut=data_mut, data_fprint=data_fprint, data_dep=data_dep, 
                      id_train=id_train, id_test=id_test, premodel_exp=premodel_exp,
                      save_path = CHECK_PATH)

In [ ]:
model_train_vis(history=model[1])

In [ ]:
SAVE_NAME = "model_cna_0801"
model[0].save(SAVE_PATH + SAVE_NAME + ".h5")
print("\n\nFull DeepDEP model saved in %s\n\n" % (SAVE_PATH + SAVE_NAME + ".h5"))